In [5]:
import pandas as pd

df = pd.read_csv("DisneylandReviews.csv", encoding="latin-1")
df

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...,...,...
42651,1765031,5,missing,United Kingdom,i went to disneyland paris in july 03 and thou...,Disneyland_Paris
42652,1659553,5,missing,Canada,2 adults and 1 child of 11 visited Disneyland ...,Disneyland_Paris
42653,1645894,5,missing,South Africa,My eleven year old daughter and myself went to...,Disneyland_Paris
42654,1618637,4,missing,United States,"This hotel, part of the Disneyland Paris compl...",Disneyland_Paris


In [14]:
df["Rating"].value_counts()

Rating
5    23146
4    10775
3     5109
2     2127
1     1499
Name: count, dtype: int64

In [15]:
undersampled_data = pd.DataFrame(columns=["Rating","Review_Text","Branch"])

undersampled_data = pd.concat([undersampled_data, df[df["Rating"]==1].sample(1499)])
undersampled_data = pd.concat([undersampled_data, df[df["Rating"]==2].sample(1499)])
undersampled_data = pd.concat([undersampled_data, df[df["Rating"]==3].sample(1499)])
undersampled_data = pd.concat([undersampled_data, df[df["Rating"]==4].sample(1499)])
undersampled_data = pd.concat([undersampled_data, df[df["Rating"]==5].sample(1499)])
df = undersampled_data
df.reset_index(inplace=True)

In [16]:
undersampled_data

,index,Rating,Review_Text,Branch,Review_ID,Year_Month,Reviewer_Location
0,17206,1,I did not think it was possible to fit that ma...,Disneyland_California,321492245.0,2015-10,United States
1,5371,1,Visited with a friend and her two early teen k...,Disneyland_HongKong,317255829.0,2015-10,Hong Kong
2,20805,1,"This was an overpriced, run down, horror of a ...",Disneyland_California,233135824.0,2014-10,United States
3,5369,1,In short. Hong Kong Disney has nothing to offe...,Disneyland_HongKong,317352620.0,2015-10,South Africa
4,19067,1,I have been to Disneyland many times over the ...,Disneyland_California,275839736.0,2015-5,United States
...,...,...,...,...,...,...,...
7490,17020,5,"Being a bit of an old, boring dad, I wasn't ne...",Disneyland_California,326835775.0,2014-12,Australia
7491,4990,5,"I love Disney, but I hate crowds. It's an horr...",Disneyland_HongKong,334381190.0,2015-12,South Africa
7492,4425,5,"Best time to visit is starting Sunday morning,...",Disneyland_HongKong,361354183.0,2016-3,Indonesia
7493,13550,5,Always fun. The park is always so well mainta...,Disneyland_California,455785252.0,2017-1,United States


In [17]:
undersampled_data["Rating"].value_counts()

Rating
1    1499
2    1499
3    1499
4    1499
5    1499
Name: count, dtype: int64

In [18]:
import tensorflow as tf
if tf.test.gpu_device_name():
  print(f"se encontro el siguiente gpu{tf.test.gpu_device_name()}")
else:
  print("aqui no hau GPU")

aqui no hau GPU


In [19]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_lg")
doc = nlp("This is a Test")
doc

This is a Test

In [20]:
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
stop_words_en = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def vectorize(texto):
    resultado = texto.lower()
    stop_words_en.append("u")
    resultado = re.sub(r"@[A-Za-z0-9]+", "", resultado)
    resultado = re.sub(r"^[A-Za-z0-9 \t]+", "", resultado)
    resultado = re.sub(r"\w+:\/\/\S+", "", resultado)
    resultado = re.sub(r"rt", "", resultado)
    resultado = re.sub(r"(\s\s\s)|(\s\s)|(\-)|(\t)", " ", resultado) 
    resultado = word_tokenize(resultado)
    resultado = [token for token in resultado if token not in stop_words_en]
    resultado = [lemmatizer.lemmatize(word) for word in resultado]
    resultado = " ".join(resultado)
    resultado = nlp(resultado).vector

    return resultado

In [22]:
df["vector"]=df["Review_Text"].apply(vectorize)

In [ ]:
from sklearn.model_selection import train_test_split
X = df["vector"]
X = np.concatenate(X,axis=0).reshape(-1,300)
y= df["Rating"]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=101,stratify=y)

In [ ]:
np.min(X_train)